In [1]:
# load .env file
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
# initialize graph and llm
import os
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI

uri = os.environ['URI']
usr = os.environ['USERNAME']
psw = os.environ['PASSWORD']

graph = Neo4jGraph(url=uri, username=usr, password=psw)

llm = ChatOpenAI(
    base_url=os.environ['BASE_URL'],
    api_key=os.environ['API_KEY'],
    model=os.environ['MODEL'],
    temperature=os.environ['TEMPERATURE'],
    max_tokens=10000,
    timeout=os.environ['TIMEOUT']
)

In [3]:
# project graph
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    os.environ["URI"],
    auth=(os.environ["USERNAME"], os.environ["PASSWORD"])
)

In [ ]:
graph_name = "communities"

if gds.graph.exists(graph_name)[0]:
    gds.graph.drop(graph_name)

G, result = gds.graph.project(
    graph_name,
    "__Entity__",
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {
                "weight": {
                    "property": "*",
                    "aggregation": "COUNT"
                }
            }
        }
    }
)

In [5]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 240
Component distribution: {'min': 2, 'p5': 2, 'max': 1063, 'p999': 1063, 'p99': 16, 'p1': 2, 'p10': 2, 'p90': 6, 'p50': 2, 'p25': 2, 'p75': 3, 'p95': 8, 'mean': 7.75}


In [6]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

ranLevels                                                                4
didConverge                                                           True
nodeCount                                                             1860
communityCount                                                         266
preProcessingMillis                                                     10
computeMillis                                                          117
postProcessingMillis                                                     2
writeMillis                                                            159
nodePropertiesWritten                                                 1860
communityDistribution    {'min': 2, 'p5': 2, 'max': 155, 'p999': 155, '...
modularities             [0.766947415145371, 0.889289446019698, 0.90885...
modularity                                                        0.909644
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [7]:
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:__Community__) REQUIRE c.id IS UNIQUE;")

[]

In [8]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 7440}]

In [9]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:MENTIONS]-(d:Document)
WITH c, count(distinct d) AS rank
SET c.community_rank = rank;
""")

[]

In [12]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,499,2.0,3.0,4.0,6.0,15.02,53
1,1,312,2.0,3.0,6.0,13.0,29.67,140
2,2,269,2.0,3.0,5.0,13.0,87.84,166
3,3,266,2.0,3.0,4.0,11.0,88.95,166


In [13]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
	whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId,
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")

In [14]:
community_info[5]

{'communityId': '0-12',
 'nodes': [{'id': 'chemical, physical, or biological threats',
   'description': None,
   'type': 'stimulus'},
  {'id': 'cell danger response',
   'description': None,
   'type': 'biological process'},
  {'id': 'old concepts of pathogenesis',
   'description': None,
   'type': 'concept'}],
 'rels': [{'start': 'cell danger response',
   'description': None,
   'type': 'IS_TRIGGERED_BY',
   'end': 'chemical, physical, or biological threats'},
  {'start': 'cell danger response',
   'description': None,
   'type': 'PERMITS_REFRAMING',
   'end': 'old concepts of pathogenesis'}]}

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """Based on the provided nodes and relationships that belong to the same graph community,
generate a natural language summary of the provided information:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [19]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

In [20]:
print(prepare_string(community_info[3]))

Nodes are:
id: autism spectrum disorders, type: disease
id: eating disorders, type: disease
id: headache disorders, type: disease
id: disability-adjusted life-years (DALYs), type: health metric
id: conduct disorders, type: disease
id: balanced diet, type: diet
id: depressive disorders, type: disease
id: other dementias, type: disease
id: neurological and mental disorders, type: disease
id: western diet, type: diet

Relationships are:
(neurological and mental disorders)-[:INCLUDE]->(autism spectrum disorders)
(neurological and mental disorders)-[:INCLUDE]->(eating disorders)
(neurological and mental disorders)-[:INCLUDE]->(headache disorders)
(neurological and mental disorders)-[:INCLUDE]->(conduct disorders)
(neurological and mental disorders)-[:INCLUDE]->(depressive disorders)
(neurological and mental disorders)-[:INCLUDE]->(other dementias)
(balanced diet)-[:ALLEVIATES]->(neurological and mental disorders)
(balanced diet)-[:PREVENTS]->(neurological and mental disorders)
(western diet

In [35]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

summaries = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

Processing communities:   0%|          | 0/811 [00:01<?, ?it/s]


APIConnectionError: Connection error.

In [ ]:
# store summaries
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})